# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [112]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [121]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [122]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's define the following parameters and decision variables:

**Parameters:**

*  *I*: Set of items to be packed.
* *J*: Set of available boxes.
* *w<sub>i</sub>*: Weight of item *i*, where *i* ∈ *I*.
* *C*: Capacity of each box.
* *N*: Maximum number of boxes allowed.

**Decision Variables:**

* *x<sub>ij</sub>*: Binary variable equal to 1 if item *i* is packed in box *j*, 0 otherwise.
* *y<sub>j</sub>*: Binary variable equal to 1 if box *j* is used, 0 otherwise.

**Objective Function:**

Minimize the number of boxes used:

```
Minimize ∑<sub>j∈J</sub> y<sub>j</sub> 
```

**Constraints:**

1. **Packing Capacity:** The total weight of items packed in a box cannot exceed its capacity.
   ```
   ∑<sub>i∈I</sub> w<sub>i</sub> * x<sub>ij</sub> ≤ C * y<sub>j</sub>,  ∀ j ∈ J
   ```

2. **Item Allocation:** Each item must be packed into exactly one box.
   ```
   ∑<sub>j∈J</sub> x<sub>ij</sub> = 1,  ∀ i ∈ I
   ```

3. **Box Limit:** The number of boxes used cannot exceed the maximum allowed.
   ```
   ∑<sub>j∈J</sub> y<sub>j</sub> ≤ N
   ```

4. **Variable Definitions:**
   ```
   x<sub>ij</sub> ∈ {0, 1},  ∀ i ∈ I, ∀ j ∈ J
   y<sub>j</sub> ∈ {0, 1},  ∀ j ∈ J
   ```

This model is a binary integer programming problem. The objective function minimizes the total number of boxes used. The first constraint enforces the box capacity, the second ensures that each item is assigned to exactly one box, and the third constraint limits the total number of boxes used. The last constraint defines the binary nature of the decision variables.


## 3. Generate the pyomo code

In [123]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [124]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data 
# (replace with your actual data)
items = ['A', 'B', 'C', 'D', 'E']
boxes = [1, 2, 3]
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12}
capacity = 25
max_boxes = 3 

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def obj_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_rule)

def allocation_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.allocation_constraint = pyo.Constraint(model.I, rule=allocation_rule)

def box_limit_rule(model):
    return sum(model.y[j] for j in model.J) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Boxes used:", pyo.value(model.obj))
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i,j]) == 1])
```

This code defines the model using Pyomo, including sets, parameters, variables, objective function, and constraints. The sample data for items, boxes, weights, capacity, and maximum number of boxes are provided as placeholders. You should replace these with your actual data. The code then uses the GLPK solver to solve the problem and displays the results.


## 4. Input problem data and try running the generated code

In [126]:
import pyomo.environ as pyo
items =  range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item

box_capacity = 9
max_boxes = 15

# Sample data 
# (replace with your actual data)
items = range(1,25)
boxes = range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item

capacity = 9
max_boxes = 15

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def obj_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_rule)

def allocation_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.allocation_constraint = pyo.Constraint(model.I, rule=allocation_rule)

def box_limit_rule(model):
    return sum(model.y[j] for j in model.J) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Display results
print(results)
print("Boxes used:", pyo.value(model.obj))
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", [i for i in model.I if pyo.value(model.x[i,j]) == 1])


Problem: 
- Name: unknown
  Lower bound: 13.0
  Upper bound: 13.0
  Number of objectives: 1
  Number of constraints: 49
  Number of variables: 600
  Number of nonzeros: 1200
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 197
      Number of created subproblems: 197
  Error rc: 0
  Time: 0.060208797454833984
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Boxes used: 13.0
Box 1: [3, 4, 9]
Box 2: [17]
Box 3: [2, 22]
Box 4: [6, 19]
Box 5: [10, 14]
Box 6: [24]
Box 7: [12, 15]
Box 8: [8, 18]
Box 9: [1, 21]
Box 10: [11, 16]
Box 11: [5, 20]
Box 12: [23]
Box 13: [7, 13]


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [127]:
print(response.text)

Let's define the following parameters and decision variables:

**Parameters:**

*  *I*: Set of items to be packed.
* *J*: Set of available boxes.
* *w<sub>i</sub>*: Weight of item *i*, where *i* ∈ *I*.
* *C*: Capacity of each box.
* *N*: Maximum number of boxes allowed.

**Decision Variables:**

* *x<sub>ij</sub>*: Binary variable equal to 1 if item *i* is packed in box *j*, 0 otherwise.
* *y<sub>j</sub>*: Binary variable equal to 1 if box *j* is used, 0 otherwise.

**Objective Function:**

Minimize the number of boxes used:

```
Minimize ∑<sub>j∈J</sub> y<sub>j</sub> 
```

**Constraints:**

1. **Packing Capacity:** The total weight of items packed in a box cannot exceed its capacity.
   ```
   ∑<sub>i∈I</sub> w<sub>i</sub> * x<sub>ij</sub> ≤ C * y<sub>j</sub>,  ∀ j ∈ J
   ```

2. **Item Allocation:** Each item must be packed into exactly one box.
   ```
   ∑<sub>j∈J</sub> x<sub>ij</sub> = 1,  ∀ i ∈ I
   ```

3. **Box Limit:** The number of boxes used cannot exceed the maximum allowed.

In [128]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data 
# (replace with your actual data)
items = ['A', 'B', 'C', 'D', 'E']
boxes = [1, 2, 3]
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12}
capacity = 25
max_boxes = 3 

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items) 
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective function
def obj_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_rule)

def allocation_rule(model, i):
    return sum(model